In [2]:
import os
from pathlib import Path
import leafmap
import rioxarray as rxr
import numpy as np
from matplotlib.colors import ListedColormap

# --------------------------
# Set up paths
# --------------------------
repo_root = Path("LULC_map")   # your local folder that will be pushed to GitHub
raster_dir = repo_root / "rasters"

filename = "TAU_LULC"
years = ["1996", "2001", "2008", "2012", "2018"]

# --------------------------
# Land cover color scheme
# --------------------------
lulc_hex = {
    1: "#419bdf",  # Water
    2: "#397d49",  # Trees
    3: "#b5c58f",  # Grass
    4: "#7a87c6",  # Wetland
    5: "#e49635",  # Crops
    6: "#68ab5f",  # Scrub
    7: "#c4281b",  # Built area
    8: "#4B0082",  # Transport
    9: "#dcd939",  # Urban green
    10: "#ccb879", # Bare land
    11: "#b3ac9f",  # Unstable ground
    12: "#510400"  # Residential
}

lulc_labels = {
    1: "Water",
    2: "Trees",
    3: "Grass",
    4: "Wetland",
    5: "Crops",
    6: "Scrub",
    7: "Built area",
    8: "Transport",
    9: "Urban green",
    10: "Bare land",
    11: "Unstable ground",
    12: "Residential"
}

# Matplotlib colormap
cmap = ListedColormap(list(lulc_hex.values()))

# --------------------------
# Compute map center from first raster
# --------------------------
first_raster_path = raster_dir / f"{filename}_1996.tif"
ds = rxr.open_rasterio(first_raster_path, masked=False).squeeze()
bounds = ds.rio.bounds()
center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# --------------------------
# Create Leafmap
# --------------------------
m = leafmap.Map(center=center, zoom=11)

# --------------------------
# Add rasters with hover
# --------------------------
for year in years:
    raster_rel_path = f"rasters/{filename}_{year}.tif"  # relative path for GitHub Pages
    raster_abs_path = os.path.join(repo_root, raster_rel_path)
    
    ds = rxr.open_rasterio(raster_abs_path, masked=False).squeeze()
    
    # Map raster codes to class names for hover
    hover_data = ds.copy()
    hover_data.values = np.vectorize(lulc_labels.get)(ds.values.astype(int))
    
    m.add_raster(
        raster_rel_path,
        colormap=cmap,
        opacity=0.7,
        nodata=0,
        vmin=1,
        vmax=len(lulc_hex),
        layer_name=f"LULC {year}",
        hover_data=hover_data
    )

# --------------------------
# Add legend
# --------------------------
m.add_legend(
    title="Land Cover",
    labels=list(lulc_labels.values()),
    colors=list(lulc_hex.values())
)

# --------------------------
# Add layer control
# --------------------------
m.add_layer_control()

m

# --------------------------
# Save HTML map (GitHub Pages–ready)
# --------------------------
html_path = os.path.join(repo_root, "map.html")
m.save(html_path)

print(f"✅ Map saved at {html_path}. Push the folder to GitHub to host online.")


OSError: Path does not exist: c:\Users\kb507\OneDrive - University of Canterbury\PhD\Paper 3 Models\Code\LULC_Map_Package\rasters\TAU_LULC_1996.tif